## Eliciting scores from LMs

Language models are trained to predict token probabilities, given some input context. This allows us to explore a number of different scoring methods.

* token-scoring: the natural ability of LMs -- assigning probabilities to tokens given context
* word-scoring: going from tokens (which could be sub-words) to word scores
* sequence-scoring: going from tokens/words to full, multi-word sequences 
* conditional-scoring: computing conditional probabilities of sequences given some input

For all these methods, we will consider a range of different scores: probabilities, log-probabilities, surprisals. In the context of sequence probabilities, we will look at differences between summing log-probabilities (equivalent to multiplying probabilities) vs. looking at log-probability per token, to account for the effect of length.

In [20]:
import torch

from minicons import scorer
from nltk.tokenize import TweetTokenizer
from torch.utils.data import DataLoader

### Different types of LMs

Autoregressive LMs: `lm.IncrementalLMScorer`

Masked LMs: `lm.MaskedLMScorer`

In [2]:
model_name = "HuggingFaceTB/SmolLM2-135M"
# model_name = "gpt2"
# model_name = "facebook/opt-125m"

# many models do not automatically insert a beggining of 
# sentence tokens when tokenizing a sequence, even though
# they were trained to do so...

if "gpt2" in model_name or "pythia" in model_name or "SmolLM" in model_name:
    BOS = True
else:
    BOS = False

lm = scorer.IncrementalLMScorer(model_name)

### Token Scoring

**Input:** I know what the lion devoured at sunrise.

**Outputs:** 
* Probabilities: $p(w_i | w_1, w_2, \dots, w_{i-1})$
* log-probabilities: $\log p(w_i | w_1, w_2, \dots, w_{i-1})$
* Surprisals: $-\log p(w_i | w_1, w_2, \dots, w_{i-1})$


In [3]:
sequences = [
    "I know what the lion devoured at sunrise.", 
    "I know that the lion devoured at sunrise."
]

Probabilities:

In [5]:
lm.token_score(
    sequences, 
    bos_token=BOS,
    prob=True,
    bow_correction=True
)

[[('<|endoftext|>', 0.0),
  ('I', 0.0020435552578419447),
  ('Ġknow', 0.017165927216410637),
  ('Ġwhat', 0.05341168865561485),
  ('Ġthe', 0.07628822326660156),
  ('Ġlion', 5.075917215435766e-05),
  ('Ġdev', 9.479006621404551e-06),
  ('oured', 0.10184427350759506),
  ('Ġat', 0.021646056324243546),
  ('Ġsunrise', 0.0006119263707660139),
  ('.', 0.1724432110786438)],
 [('<|endoftext|>', 0.0),
  ('I', 0.0020435552578419447),
  ('Ġknow', 0.017165927216410637),
  ('Ġthat', 0.2915891110897064),
  ('Ġthe', 0.13011601567268372),
  ('Ġlion', 8.081357373157516e-05),
  ('Ġdev', 7.503021333832294e-05),
  ('oured', 0.32275477051734924),
  ('Ġat', 0.0006744185811839998),
  ('Ġsunrise', 8.538350084563717e-05),
  ('.', 0.07048039138317108)]]

log probabilities/surprisals:

In [11]:
lm.token_score(
    sequences, 
    bos_token=BOS,
    surprisal=True
)

[[('<|endoftext|>', 0.0),
  ('I', 5.686635971069336),
  ('Ġknow', 4.515291213989258),
  ('Ġwhat', 2.960165023803711),
  ('Ġthe', 2.5910110473632812),
  ('Ġlion', 9.441681861877441),
  ('Ġdev', 12.020919799804688),
  ('oured', 1.3213729858398438),
  ('Ġat', 4.788764953613281),
  ('Ġsunrise', 7.406002998352051),
  ('.', 1.361612319946289)],
 [('<|endoftext|>', 0.0),
  ('I', 5.686635971069336),
  ('Ġknow', 4.515291213989258),
  ('Ġthat', 1.2616539001464844),
  ('Ġthe', 2.0563831329345703),
  ('Ġlion', 9.177278518676758),
  ('Ġdev', 9.753374099731445),
  ('oured', 1.1199226379394531),
  ('Ġat', 7.276799201965332),
  ('Ġsunrise', 9.404158592224121),
  ('.', 2.3709983825683594)]]

### Word scoring

Same metrics, but logprobs for words that are split into tokens are summed---e.g., `devoured` is split into `dev + oured`. However, here you have to provide the word tokenizer yourself. We will use `nltk`'s `TweetTokenizer()` as an example

In [12]:
word_tokenizer = TweetTokenizer().tokenize

In [13]:
lm.word_score_tokenized(
    sequences, 
    bos_token=BOS, 
    tokenize_function=word_tokenizer,
    surprisal=True,
    base_two=True
)

[[('I', 8.204081535339355),
  ('know', 6.514188289642334),
  ('what', 4.270615577697754),
  ('the', 3.7380387783050537),
  ('lion', 13.621467590332031),
  ('devoured', 19.248859405517578),
  ('at', 6.908727645874023),
  ('sunrise', 10.684603691101074),
  ('.', 1.9643913507461548)],
 [('I', 8.204081535339355),
  ('know', 6.514188289642334),
  ('that', 1.8201818466186523),
  ('the', 2.966733694076538),
  ('lion', 13.24001407623291),
  ('devoured', 15.686850547790527),
  ('at', 10.498202323913574),
  ('sunrise', 13.567333221435547),
  ('.', 3.4206275939941406)]]

### Sequence scoring

**Input:** batch of sentences

**Outputs:** scores indicating how likely each sequence is. There are multiple methods for doing this though:

* summed log-probs (equivalent to joint probability, computed using the product rule)
* log-prob per token

In [14]:
sequences = [
    "The keys to the cabinet are on the table.",
    "The keys to the cabinet is on the table."
]

log-prob per token (default behavior):

In [15]:
lm.sequence_score(sequences, bos_token=BOS)

[-3.6746392250061035, -4.0424675941467285]

summed log-probs:

summing is done by using the `reduction` argument, which takes a function

In [27]:
lm.sequence_score(
    sequences, 
    bos_token=BOS, 
    reduction=lambda x: x.sum().item()
)

[-36.74639129638672, -40.42467498779297]

Here, the lambda function is a concise way of defining a function, here this is equivalent to taking the torch tensor consisting of the model elicited log-probabilities and reduces it row-wise by summing, and extracting the item (as opposed to keeping it as a `tensor`). For example:

In [49]:
x = torch.tensor([0.223234, 0.443257, 0.364343], dtype=torch.double)
sum_func = lambda x: x.sum().item()

sum_func(x)

1.030834

### log-prob of full sequence (summing) vs. log-prob per token (avg)

Usually, the two metrics show similar qualitative trends, especially for minimal pair comparisons. However there are certain cases where log-prob per token is a better metric. This is because the summed log prob metric for a sentence might be lower simply because it is longer (contain more tokens)--since it involves a more number of multiplications between word-probabilities, each of which is a number lower than 1.

The following pair illustrates this issue:

1. These casseroles disgust Mrs. O'leary
2. *These casseroles disgusts Kayla

In [42]:
stimuli = [
    "These casseroles disgust Mrs. O'leary", # longer but grammatical
    "These casseroles disgusts Kayla" # shorter but ungrammatical
]

In [43]:
# sum
lm.sequence_score(stimuli, bos_token=BOS, reduction=lambda x: x.sum().item())

[-61.54802703857422, -56.2628173828125]

In [44]:
lm.sequence_score(stimuli, bos_token=BOS)

[-6.1548027992248535, -8.037545204162598]

### Conditional LM scoring

This follows the same principle as sequence scoring, but allows you to separate the prefix and the continuation. Like sequence scoring, this method also allows for different reduction methods

In [50]:
# Lake and Murphy (2023) / Murphy (1988)
# "are cooked in a pie" is an emergent property of sliced apples

prefix = ["Sliced apples", "Apples", "Sliced things"]
continuation = ["are cooked in a pie."] * 3

In [51]:
lm.conditional_score(prefix, continuation, bos_token=BOS)

[-3.236760377883911, -3.915409803390503, -3.4075047969818115]